## <center> Projet 3 - Parcours Machine Learning :<br/>Consommation électrique de la ville de Seattle </center>

<p style="text-align:center;">Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près aux émissions des bâtiments non destinés à l’habitation.</p>

<center><img src="image/logo_seattle.png" alt="logo_seattle" width="500" /></center>

##### Les données

Les données de consommation sont [à télécharger ici](https://www.kaggle.com/city-of-seattle/sea-building-energy-benchmarking#2015-building-energy-benchmarking.csv).

##### Problématique de la ville de Seattle

<p style="text-align:justify;">Des relevés minutieux ont été effectués par vos agents en 2015 et en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments pour lesquels elles n’ont pas encore été mesurées.

Votre prédiction se basera sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction..)

Vous cherchez également à évaluer l’intérêt de l’"ENERGY STAR Score" pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe.</p>

##### Votre mission

Vous sortez tout juste d’une réunion de brief avec votre équipe. Voici un récapitulatif de votre mission :

- Réaliser une courte analyse exploratoire.
- Tester différents modèles de prédiction afin de répondre au mieux à la problématique.

Avant de quitter la salle de brief, Douglas, le project lead, vous donne quelques pistes, et erreurs à éviter :

>L’objectif est de te passer des relevés de consommation annuels (attention à la fuite de données), mais rien ne t'interdit d’en déduire des variables plus simples (nature et proportions des sources d’énergie utilisées). 
>
>Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).
>
>Mets en place une évaluation rigoureuse des performances de la régression, et optimise les hyperparamètres et le choix d’algorithme de ML à l’aide d’une validation croisée.
>

#### Let's go :

In [1]:
###################################################################################################
####################                       Import librairies                    ###################
###################################################################################################
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from math import *
from numpy import isnan
from global_functions_variables import *
import altair as alt
from vega_datasets import data

#### Les fichiers CSV :

<p>Les relevés énergétiques sont stockés dans 2 fichiers csv :</p>

- Les relevés de 2015
- Les relevés de 2016

<p>On remarque rapidement que les deux bases de données n'ont pas le même nombre de colonnes.
Une analyse des différences montre les faits suivants :</p>

1. Les variables de localisation du fichier de 2016 :<br>
    *Latitude*,<br>
    *Longitude*,<br>
    *City*,<br>
    *Address*,<br>
    *zipCode*,<br>
    *State*<br> sont **compressées** dans une seule variable *Location* dans le fichier de 2015

2. Les variables du fichier 2015 :<br> 
     *OtherFuelUse(kBtu)*,<br> 
     *2010 Census Tracts*,<br> 
     *Seattle Police Department Micro Community Policing Plan Areas*,<br> 
     *City Council Districts*,<br> 
     *SPD Beats*,<br>
     *Zip Codes*,<br>non présentes en 2016, sont **vides, faussées ou sans rapport avec la consommation d'énergie**
    
3. Les dernières différences de variables représentent **les mêmes données sous la même métrique** mais **sous un autre nom**.

<p>Pour rassembler ces deux bases de données il faut donc :</p>

1. Décompresser la variable *Location* de 2015 et sauvegarder ses valeurs dans de nouvelles colonnes de même noms que la version 2016

2. Retirer les colonnes non pertinentes du DataFrame de 2015

3. Renommer les variables restantes pour s'adapter aux variables de 2016

Une fois ces étapes effectuées, les deux DataFrames pourront être combinés afin de traiter les données de 2015 et de 2016 simultanément.

In [2]:
###################################################################################################
####################          Load and merge CSV files from 2015 and 2016       ###################
###################################################################################################

#read csv file and stock it in dataframe
data_2015 = pd.read_csv(r'Data/2015-building-energy-benchmarking.csv',
                        sep=',', encoding='utf-8', na_values='')
data_2016 = pd.read_csv(r'Data/2016-building-energy-benchmarking.csv',
                        sep=',', encoding='utf-8', na_values='')
print(f"The 2016 CSV file had {data_2016.shape[1]}\
 fields and the 2015 CSV file had {data_2015.shape[1]} fields")
    
# 1.Re-Organize location variables - Dezip latitude, longitude ect..
decompressed = {'Latitude': [],
                'Longitude' : [],
                'City' : [],
                'Address' : [],
                'ZipCode' : [],
                'State': []}

for loc in data_2015['Location']:
    loc_dict = eval(loc)
    address_dict = eval(loc_dict['human_address'])
    decompressed['Latitude'].append(round(float(loc_dict['latitude']),4))
    decompressed['Longitude'].append(round(float(loc_dict['longitude']),4))
    decompressed['City'].append(address_dict['city'])
    decompressed['Address'].append(address_dict['address'])
    decompressed['ZipCode'].append(address_dict['zip'])
    decompressed['State'].append(address_dict['state'])
data_2015 = pd.concat([data_2015.drop('Location',axis=1), pd.DataFrame(decompressed)],axis=1)

# 2. Remove variables only present in 2015 AND empty, irrelevant or wrong :
data_2015.drop(["OtherFuelUse(kBtu)",'2010 Census Tracts',
                'Seattle Police Department Micro Community Policing Plan Areas',
                'City Council Districts',"SPD Beats", 'Zip Codes'], axis=1, inplace=True)

# For "GHGEmissions(MetricTonsCO2e) --> TotalGHGEmissions 
#we have checked beforehand that 2015 and 2016 dataset use the same metrics !

# 3. Rename 2015 variable to their 2016 names
data_2015.rename(columns={"Comment" : "Comments",
                          "GHGEmissions(MetricTonsCO2e)" : "TotalGHGEmissions",
                         "GHGEmissionsIntensity(kgCO2e/ft2)" : "GHGEmissionsIntensity"
                         },inplace=True)

# Finally, combine both into a 2015-2016 dataframe
if len(set(data_2016.columns).symmetric_difference(set(data_2015.columns))) == 0 :
    print("Both datasets have the same number of columns, it could be concatenate now")
    data_global = pd.concat([data_2015, data_2016], axis=0)
else:
    print("Something gone wrong")

data_global.reset_index(drop=True, inplace=True)
print(f"The final shape of 2015-2016 dataset is {data_global.shape}")

The 2016 CSV file had 46 fields and the 2015 CSV file had 47 fields
Both datasets have the same number of columns, it could be concatenate now
The final shape of 2015-2016 dataset is (6716, 46)



---

### Analyse des variables:

<p>Notre DataFrame possède 46 variables qu'il faut étudier pour déduire de leur pertinence et de leur impact sur notre projet. Pour cela, observer leur type, leur valeurs uniques, leur valeurs manquantes ou encore leur métrique apporte de précieuses informations.</p> 

#### Les variables de localisation:

<p>Les données de localisation sont importantes pour visualiser nos résultats sur une carte, ou étudier la consommation par quartier. L'emplacement d'un bâtiment est-il corrélé à sa consommation énergétique? Pour le découvrir, il faut conserver ces données. </p>

- *Latitude* et *Longitude* : <br/> Axe Nord-Sud et Ouest-Est des coordonnées terrestres, permet une localisation précise et pratiquement unique. Utilisable pour détecter les doublons.<br/>**à conserver**

- *Neighborhood* et *CouncilDistrictCode* : <br/> Chacune de ces variables représente des zones géographiques, en les combinant, la granularité sera plus fine. Les valeurs de *Neighborhood* ne sont pas standardisées (orthographe alternative etc...). Regroupement sous une seule variable à envisager.<br/>**à conserver, standardiser et reformater**

- *Address* :<br/> Représentation "humaine" de la position du bâtiment, ne sera pas utilisable par nos systèmes, d'autant plus que de meilleures informations de positionnement existent. Utilisable pour l'identification.<br/> **à reformater dans une variable d'identification**

- *State*, *ZipCode* et *City* :<br/>  *State* et *City* n'ont qu'une seule valeur possible dans ce jeu de données. *ZipCode* n'est pas pertinent.<br/>**à retirer**


#### Les variables de description du bâtiment:

<p>Rappel du projet :</p>

>Votre prédiction se basera sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction..)

<p>Les informations relatives aux bâtiments sont les données de base de notre système, sur lesquels on s'appuieras pour prédire la consommation. La récupération de ces données est peu coûteuse.</p> 

- *OSEBuildingID*, *TaxParcelIdentificationNumber* et *PropertyName* :<br/>Variables d'identification, identifient les bâtiments ( ou leur propriétaire dans le cas de *TaxParcelIdentificationNumber*) par un numéro ou un nom. Utiles pour manipuler les données et détecter les doublons <br/>**à reformater dans une variable d'identification**

- *YearBuilt*, *NumberofBuildings* et *NumberofFloors* :<br/>Variables descriptives du bâtiments.<br/>**à conserver**

- *PropertyGFATotal*, *PropertyGFAParking*, *PropertyGFABuilding(s)* :<br/>GFA signifie Gross Floor Area : la superficie au sol du bâtiment en incluant les murs extérieurs. Ces variables représentent la superficie au sol du bâtiment avec ou sans prendre en compte le parking. Une forte corrélation est attendue entre ces variables, l'analyse exploratoire les réduira. Presque 50% des individus n'ont qu'un seul type d'usage.<br/> **à conserver**

- *BuildingType*, *PrimaryPropertyType*, *ListOfAllPropertyUseTypes* et autres :<br/>Type d'usage du bâtiment selon différentes granularités. Si le bâtiment a plus d'un seul type d'usage, la surface au sol allouée aux trois premiers types d'usage est disponible dans les variables *LargestPropertyUseTypeGFA*, *SecondLargestPropertyUseTypeGFA* et *ThirdLargestPropertyUseTypeGFA*. Cette méthode de stockage d'information provoque un fort taux de valeurs manquantes. De plus, une redondance est présente entre *ListOfAllPropertyUseTypes* et les trois variables *LargestPropertyUseType*, *SecondLargestPropertyUseType*, *ThirdLargestPropertyUseType*. Des modifications sur ces variables sont donc à prévoir.<br/>**à reformater et conserver** sauf *ListOfAllPropertyUseTypes* **à retirer pour redondance**

#### Les variables énergetiques:

<p>Rappel du projet :</p>

>Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments pour lesquels elles n’ont pas encore été mesurées.

<p>Les variables énergetiques sont les données que notre modélisation doit prédire. Le recensement de ses données est couteux et à éviter. Les données conservées de cette catégorie servirons de vérification lors de l'entrainement de nos modèle supervisés</p> 
       
- *YearsENERGYSTARCertified* et *ENERGYSTARScore*: <br/> Un des objectifs du projet est de déterminer la pertinence du score ENERGYSTAR. Pour autant *YearsENERGYSTARCertified* a un fort taux de valeurs manquantes et est redondants avec le score<br/> *ENERGYSTARScore* **à conserver**, *YearsENERGYSTARCertified* **à retirer**
       
- *SteamUse(kBtu)*, *NaturalGas(kBtu)*, *Electricity(kBtu)* et *SiteEnergyUse(kBtu)*: <br/> Les mesures des différents types d'énergie et leur total en unité thermique britannique. Notre modèle n'a pas pour but de déterminer quel type d'énergie est consommée, seul le total est conservé<br/> *SiteEnergyUse(kBtu)* **à conserver**, les autres **à retirer**       

- *NaturalGas(therms)* et *Electricity(kWh)*: <br/> Ces variables sont des doublons et leur métrique n'est pas le standard dans notre base de données<br/> **à retirer**

- *TotalGHGEmissions*: <br/> Le modèle doit également prédire l'émissions de gaz à effet de serre, la variable est conserver pour l'apprentissage<br/> **à conserver**
       
- *SiteEUI(kBtu/sf)*, *GHGEmissionsIntensity* et *SourceEUI(kBtu/sf)*: <br/> Ces variables sont le résultat d'un calcul entre la surface au sol du bâtiment et sa consommation. Elles découlent directement de la variable à prédire<br/> **à retirer**
       
- *SiteEUIWN(kBtu/sf)*, *SourceEUIWN(kBtu/sf)* et *SiteEnergyUseWN(kBtu)*: <br/> WN : Weather Normalized : Il s'agit des variables précédentes normalisée par les données météorologiques des 30 dernières années. Notre base de données ne possède pas d'informations relatives à la météo ou au climat, pourtant il s'agit d'une donnée essentielle en prédiction de consommation énergétique. Prédire une consommation relative au climat est selon moi bien plus pertinent que de prédire la consommation brute en supposant une météo stable. Pour autant, pour suivre l'intitulé du projet, ces informations (dont la formule est inconnue) ne seront pas utilisées <br/> **à retirer**
       
<p>Un énorme biais se remarque par la non prise en compte de variable temporelle. La consommation électrique d'une année sur l'autre diffèrent due à de nombreux facteurs qui sont mis de côté dans notre étude. Le résultat finale s'en retrouvera affaiblie et nos prédictions seront inexactes. Cela est d'autant plus d'actualité que le changement climatique a profondement bouleversés la stabilité météorologique qui est supposé dans ce modèle erroné.</p>
       
#### Autres variables :

- *Outlier* et *Comments*: <br/> Ces variables ont un taux de valeurs manquantes extremement haut (supérieur à 95%) et leurs valeurs présentes ne sont pas exploitables.<br/>**à retirer**

- *DefaultData* : <br/> Cette variable précise si au moins une des variables de l'individu a conservé des valeurs par default. Cela représente une mesure de fiabilité du relevé mais n'apporte pas d'informations utiles à notre projet. N'implique pas forcément une corruption de l'individu, pas de selection à effectuer sur ce critère<br/>**à retirer**

- *DataYear* :<br/> Seules deux valeurs possibles pour cette variable : 2015 et 2016. Notre modélisation n'a pas de dimension temporelle : l'étude ne prends pas en compte les facteurs spécifiques à une année (météo).La variable ne sera pas utilisable.<br/> **à retirer**

- *ComplianceStatus* :<br/> Représente si l'individu a atteint les objectifs demandé par l'analyse comparative du rendement énergétique, la variable est donc déduite de la consommation énergétique, il ne s'agit pas d'une donnée exploitable<br/>**à retirer**

#### Conclusion de l'analyse :

<p>Selon l'analyse les tâches suivantes seront à effectuées sur les variables:</p>

 **Retirer du dataset les variables:** *ComplianceStatus*, *DataYear*, *DefaultData*, *Outlier*, *Comments*, *SiteEUIWN(kBtu/sf)*, *SourceEUIWN(kBtu/sf)*, *SiteEnergyUseWN(kBtu)*, *SiteEUI(kBtu/sf)*, *ListOfAllPropertyUseTypes*, *GHGEmissionsIntensity*, *SourceEUI(kBtu/sf)*,*NaturalGas(therms)*, *Electricity(kWh)*, *SteamUse(kBtu)*, *NaturalGas(kBtu)*, *Electricity(kBtu)*, *YearsENERGYSTARCertified*, *ZipCode*, *State* et *City*

 **Reformater les variables suivantes :** *BuildingType*, *PrimaryPropertyType*, *ListOfAllPropertyUseTypes*, *LargestPropertyUseTypeGFA*, *SecondLargestPropertyUseTypeGFA*, *ThirdLargestPropertyUseTypeGFA*, *LargestPropertyUseType*, *SecondLargestPropertyUseType* et *ThirdLargestPropertyUseType*

 **Créer une variable d'identification avec :** *OSEBuildingID*, *TaxParcelIdentificationNumber*, *PropertyName* et *Address*
 
 **Créer une variable de zone géographique avec :** *Neighborhood*, *ZipCode* et *CouncilDistrictCode*

 **Déterminer quelles variables sont les plus pertinentes entre :** *PropertyGFATotal*, *PropertyGFAParking*, *PropertyGFABuilding(s)*

In [3]:
###################################################################################################
####################                   Reformating variables                    ###################
###################################################################################################

#Keeping a DataFrame with all informations, it could be usefull
df = data_global

#Reformate values spell check PrimaryPropertyType values - remove '/n' at the end
type_names = df['PrimaryPropertyType'].apply(lambda x : x.replace('\n',''))
df.loc[:, 'PrimaryPropertyType'] = type_names 

df['LargestPropertyUseType'].fillna(df['PrimaryPropertyType'], inplace=True)
df.loc[(df['LargestPropertyUseTypeGFA'].isna() & df['SecondLargestPropertyUseType'].isna()),
       'LargestPropertyUseTypeGFA'] = df['PropertyGFABuilding(s)']
df['LargestPropertyUseType'].replace(['Mixed Use Property'], ['Other'] ,inplace=True)
df['ListOfAllPropertyUseTypes'].fillna(df['LargestPropertyUseType'],
                                                #inplace=True)
df['LargestPropertyUseType'].replace(rename_usetype.keys(),
                                              rename_usetype.values(), inplace=True)
df['SecondLargestPropertyUseType'].replace(rename_usetype.keys(),
                                                    rename_usetype.values(), inplace=True)
df['ThirdLargestPropertyUseType'].replace(rename_usetype.keys(),
                                                   rename_usetype.values(), inplace=True)
df['PrimaryPropertyType'].replace(rename_usetype.keys(),
                                           rename_usetype.values(), inplace=True)

#Remove parking from PropertyUsage - Check with 'PropertyGFAParking'
#TODO

#
#

#Format latitude and longitude to the center of seattle.
seattle_lat = 47.6062095
seattle_long = -122.3320708

#Before, apply it to a map ? 
#Note that 2016 localisation are more accurate
new_latitude = df['Latitude'].apply(lambda x : seattle_lat - x)
new_longitude = df['Longitude'].apply(lambda x : seattle_long - x)
df.loc[:, 'Latitude'] = new_latitude
df.loc[:, 'Longitude'] = new_longitude

#Zones géographique categorie
neighbor_names = df['Neighborhood'].apply(lambda x : x.upper())
df.loc[:, 'Neighborhood'] = neighbor_names
df['Neighborhood'].replace('DELRIDGE NEIGHBORHOODS',
                                    'DELRIDGE', inplace=True)
district = []
for i, val in df[['CouncilDistrictCode', 'Neighborhood']].T.items():
    d = val['CouncilDistrictCode']
    n = val['Neighborhood']
    if d == 3 and 'EAST' in n:
        n = 'EAST'
    elif d == 3 :
        n = 'CENTRAL'
    elif d == 4 and 'NORTH' in n:
        n = 'NORTHEAST'
    elif d == 6 and 'LAKE UNION' == n:
        n = 'NORTHWEST'
    elif d == 5 and 'NORTHEAST' == n:
        n = 'NORTH'
    elif d == 2 and 'EAST' == n:
        n = 'SOUTHEAST'
    elif d == 7 and 'EAST' == n:
        n = 'LAKE UNION'
    elif d == 1 and 'GREATER DUWAMISH' == n:
        n = 'DELRIDGE'
    district.append('District_'+str(d)+'_'+n)
pd.Series(district).value_counts()
#df.drop(['CouncilDistrictCode', 'Neighborhood'], axis=1, inplace=True)
df["Zone"] = pd.Series(district)
#Vérifier sur une carte la pertinence de ma notation

#Reformating identification
description = df.agg(lambda x: f"{x['OSEBuildingID']} : {x['PropertyName']} at {x['Address']}", axis=1)
df.loc[:, "Description"] = description

#### Nettoyage des données :

##### Supprimer les bâtiments résidentiels:

<p>Dans l'intitulé du projet il est dit :</p>

>Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près aux émissions **des bâtiments non destinés à l’habitation.**

<p>Les bâtiments résidentiels doivent être supprimer de notre DataFrame. Les valeurs de *BuildingType* et *PrimaryPropertyType* sont étudiées, il s'y trouve trois valeurs représentant des bâtiment résidentiels :</p>

 - Multifamily MR (5-9)
 - Multifamily LR (1-4)
 - Multifamily HR (10+)
 - Low-Rise Multifamily
 
<p>Tout individus ayant ces valeurs dans le champs *BuildingType* est donc retirer de notre DataFrame</p>

<p>Dans le même temps, les variables :</p> 

- *BuildingType*, 
- *PrimaryPropertyType*,
- *ListOfAllPropertyUseTypes*,
- *LargestPropertyUseType*,
- *SecondLargestPropertyUseType*,
- *ThirdLargestPropertyUseType* 

<p>seront reformatées</p>

En analysant de plus près les données de ces variables les faits suivants sont visibles :

- *ListOfAllPropertyUseTypes*

La liste des types d'usage *ListOfAllPropertyUseTypes* n'est pas ordonnée selon la hierarchie définie par *LargestPropertyUseType*, *SecondLargestPropertyUseType* et *ThirdLargestPropertyUseType*. La taille maximum de cette liste est de 13 mais seuls 8,6% des individus ont plus de 3 usages. Presque la moitié des individu (49,5%), n'ont qu'un seul et unique usage. Conserver les informations relatives aux usages après le troisième plus important est donc peu pertinent. *ListOfAllPropertyUseTypes* ne semble pas nécessaire et est donc supprimée.

- Les parkings

La taille des parking est déjà prise en compte dans la variable *GFAParking*, il est donc redondant de retrouver 'parking' dans la liste des usages du bâtiment. Nous allons retirer cette usage de nos listes lorsqu'on décompte plus d'un seul usage.

In [4]:
###################################################################################################
####################                   Réduction des individus                  ###################
###################################################################################################

# Removing residentials buildings from dataset
resi_buildings = ['Multifamily MR (5-9)', 'Multifamily LR (1-4)', 'Multifamily HR (10+)']
df = df.loc[~df['BuildingType'].isin(resi_buildings)]
df = df.loc[df['PrimaryPropertyType'] != 'Low-Rise Multifamily']

# Removing individual without energy use or GHG emissions count
# is it needed ?
df = df.loc[~df['TotalGHGEmissions'].isna()]
df = df.loc[~df['SiteEnergyUse(kBtu)'].isna()]


# Removing duplicates from dataset
# Etudier les duplicate afin de montrer l'intérêt 
#d'une étude temporelle prenant en compte les années.
# Pour le moment soyons basique :
df.drop_duplicates(subset=['OSEBuildingID'], inplace=True)
#Print les écarts entre 2016 et 2015 ?

In [5]:
###################################################################################################
####################                   Reduction des variables                  ###################
###################################################################################################

# Removing variable from dataset
variables_indésirables = ['ComplianceStatus', 'DataYear', 'DefaultData', 'Outlier', 'Comments',
                          'SiteEUIWN(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUseWN(kBtu)',
                          'SiteEUI(kBtu/sf)', 'GHGEmissionsIntensity', 'SourceEUI(kBtu/sf)',
                          'NaturalGas(therms)', 'Electricity(kWh)', 'SteamUse(kBtu)',
                          'NaturalGas(kBtu)', 'Electricity(kBtu)', 'YearsENERGYSTARCertified',
                          'ZipCode', 'State', 'City', 'ListOfAllPropertyUseTypes']

#These variables have been reformated :
variables_indésirables.extend(['Address','OSEBuildingID', 'PropertyName', 'CouncilDistrictCode',
                               'Neighborhood', 'TaxParcelIdentificationNumber'])

df.drop(variables_indésirables, axis=1, inplace=True)

In [23]:
###################################################################################################
####################       Recherche d'erreurs et de valeurs aberrantes         ###################
###################################################################################################


#valeur aberrante : row 6614 l'université de washington 111 bâtiments différents,
#une années de construction trouble ( 1861 selon wiki, 1900 selon le dataset)
#df.drop(6614, inplace=True)

df.loc[1371,'NumberofFloors'] = 2 #erreur CHINESE BAPTIST CHURCH don't have 99 floors but only 2

#corrige les erreurs basique de GFA
for i, gfa in df[[x for x in df.columns if 'GFA' in x]].copy().T.items():
    for c, val in gfa.items() :
        if val < 0 and val > -10 :
            df.loc[i,c] = 0
        elif val < 0 :
            df.loc[i,c] = val*-1
    if gfa['PropertyGFATotal'] != gfa['PropertyGFAParking'] + gfa['PropertyGFABuilding(s)'] :
        if gfa['PropertyGFAParking'] ==  gfa['PropertyGFATotal'] + gfa['PropertyGFABuilding(s)']:
            df.loc[i, 'PropertyGFAParking'] = gfa['PropertyGFATotal']
            df.loc[i, 'PropertyGFATotal'] = gfa['PropertyGFAParking']
        else:
            df.loc[i, 'PropertyGFATotal'] = gfa['PropertyGFAParking']+gfa['PropertyGFABuilding(s)']
            
            
print(data_global['ListOfAllPropertyUseTypes'].loc[data_global['PropertyGFAParking']==-2])
deja_vu = ['BuildingType', 'PrimaryPropertyType', 'YearBuilt', 'NumberofBuildings',
          'NumberofFloors', 'PropertyGFATotal', 'PropertyGFABuilding(s)', 'PropertyGFAParking',
          ]
for c in df.columns:
    if c not in deja_vu :
        print(df[c].describe())
        print(df[c].isna().sum())
        print()


3136    Bank Branch, Office
Name: ListOfAllPropertyUseTypes, dtype: object
count       1694
unique        47
top       Office
freq         490
Name: LargestPropertyUseType, dtype: object
0

count    1.694000e+03
mean     9.309461e+04
std      1.589225e+05
min      5.872000e+03
25%      2.548025e+04
50%      4.348400e+04
75%      9.039900e+04
max      1.719643e+06
Name: LargestPropertyUseTypeGFA, dtype: float64
0

count         832
unique         36
top       Parking
freq          322
Name: SecondLargestPropertyUseType, dtype: object
862

count       832.000000
mean      36524.926683
std       69741.398263
min           0.000000
25%        5435.000000
50%       12280.500000
75%       31111.750000
max      686750.000000
Name: SecondLargestPropertyUseTypeGFA, dtype: float64
862

count        343
unique        32
top       Office
freq          48
Name: ThirdLargestPropertyUseType, dtype: object
1351

count       343.000000
mean      14311.714869
std       36721.097656
min           0.00000

#### Nettoyage des données :

##### Supprimer les doublons:
 
 **Supprimer les doublons de bâtiments** et choisir quelles valeurs conserver
 
 **Supprimer les individus sans informations** s'ils existent
 
 **Rechercher les valeurs aberrantes et les corriger**
 
 **Traiter les valeurs NaN**

Nous allons regrouper les données d'identification des bâtiments afin qu'elles ne perturbent pas nos modélisations mais restent utilisables.

identification_features = ['OSEBuildingID', 'PropertyName', 'Address', 'ZipCode']
data_identification = data[identification_features]
data.drop(identification_features, axis=1, inplace = True)


Le but de notre programme est de supprimer les relevés couteux pour les années à venir. Nous allons donc exclure toutes les données de relève de notre dataset.

In [7]:
#Doublons
df.drop_duplicates(subset=['OSEBuildingID'], inplace=True)
list_doublons = []
for i, row in df.T.items():
    if i not in big_set :
        big_set = set()
        address = row['Address']
        Bldg_ID = row['OSEBuildingID']
        Owner_ID = row['TaxParcelIdentificationNumber']
        name = row['PropertyName']
        lat = row['Latitude']
        long = row['Longitude']
        same_loc = df.loc[(df['Latitude'] == lat) & (df['Longitude'] == long) & (df['TaxParcelIdentificationNumber'] == Owner_ID)]# & (df['Address'] == address)]
        if same_loc.shape[0] > 1 :
            big_set.update(same_loc.index.values)
        if len(list_doublons) == 0 and len(big_set) != 0 :
            list_doublons.append(big_set)
        else :
            for x in list_doublons.copy():
                if big_set.intersection(x):
                    #if some elements are intercorrelated, merge the group
                    list_doublons[list_doublons.index(x)] = set.union(x, big_set)
                    big_set = set() # avoid adding duplicate
                    break
            if len(big_set) != 0 :
                # add a new correlated group only if no intersection 
                list_doublons.append(big_set)

for doubles in list_doublons:
    for id_d in list(doubles):
        print(df.loc[id_d,'PropertyName'], " and ", df.loc[id_d,'DataYear'])
    print('\n')

KeyError: Index(['OSEBuildingID'], dtype='object')

In [ ]:
###################################################################################################
####################                   Traitement des données                   ###################
###################################################################################################

# Traitement des données 1 : Removing residential building from dataset
resi_buildings = ['Multifamily MR (5-9)', 'Multifamily LR (1-4)', 'Multifamily HR (10+)']
df = df.loc[~df['BuildingType'].isin(resi_buildings)]


for column in df.columns:
    if df[column].nunique()<20:
        print('Colonne {}, valeurs uniques :\n{}\n'.format(column, df[column].unique()))
    else:
        print('Colonne {}, {} valeurs uniques'.format(column, df[column].nunique()))




